In [1]:
import nltk
import pandas as pd
import numpy as np

In [2]:
train_data=pd.read_csv("twitter_train.csv")
train_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.679000e+17,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,17-02-2015 20:16,Washington D.C.,Atlantic Time (Canada)
1,5.699890e+17,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,23-02-2015 14:36,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,5.680890e+17,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,18-02-2015 08:46,Illinois,Central Time (US & Canada)
3,5.689280e+17,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,20-02-2015 16:20,NaN,Atlantic Time (Canada)
4,5.685940e+17,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,19-02-2015 18:13,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,5.699340e+17,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,23-02-2015 10:58,"ohio,panama",NaN
10976,5.685640e+17,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,19-02-2015 16:13,Brooklyn,Eastern Time (US & Canada)
10977,5.696440e+17,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,22-02-2015 15:43,"St. Augustine, Florida",NaN
10978,5.688650e+17,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,20-02-2015 12:09,NaN,NaN


In [3]:
del train_data['negativereason_gold']
del train_data['airline_sentiment_gold']
del train_data['tweet_coord']
del train_data['tweet_id']
del train_data['name']
del train_data['tweet_created']
del train_data['user_timezone']

In [4]:
#Converting string values to numbers in airline column
airline_num={'Southwest':1, 'US Airways':2, 'Delta':3, 'American':4, 'United':5, 'Virgin America':6}
train_data['airline']=[airline_num[item] for item in train_data['airline']]

In [5]:
del train_data['tweet_location']

In [ ]:
#Creating doc in the format:- [[comments],[sentiment type]]
train_docs=()
for i in range(len(train_data)):
    train_docs.append((train_data['text'][i],train_data['airline_sentiment'][i]))

In [ ]:
#Cleaning text in docs:-

#(1)Creatting set of stopwords and punctuations
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)

#(2)Function to get simple pos form from WordNet dataset
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
#(3)Importing Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

#(4)Tokenizing function
from nltk.tokenize import word_tokenize

for i in range(len(train_docs)):
    train_docs[i][0]=word_tokenize(str(train_docs[i][0]))
    
    
#(5)Cleaning Function
from nltk import pos_tag
def clean(words):
    output_words=[]
    for w in words:
        w=w.lower()
        if w not in stops:
            pos=pos_tag([w])
            cleaned_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(cleaned_word.lower())
    return output_words

In [ ]:
for i in range(len(train_docs)):
    train_docs[i][0]=clean(train_docs[i][0])

In [ ]:
#Creating combined worset
allWords=[]
for i in range(len(train_docs)):
    allWords+=train_docs[i][0]

In [ ]:
#Getting frequency distribution for all words
import nltk
freq=nltk.FreqDist(allWords) #Storing the object obtained in another object called freq
common=freq.most_common(3000) 
#Array of tuples obtained
features= [i[0] for i in common]

In [ ]:
#Creating dictionary with words and whether they are present or not
def get_feature_dict(words):
    current_features={}
    wordSet=set(words)
    for w in features:
        current_features[w]= w in wordSet
    return current_features

In [ ]:
o=get_feature_dict(train_docs[0][0])
o

In [ ]:
train_docs[0][1]

In [ ]:
x_train=[]
for i in range(len(train_docs)):
    x_train.append((get_feature_dict(train_docs[i][0]), train_docs[i][1]))
x_train

In [ ]:
from nltk import NaiveBayesClassifier
classifier=NaiveBayesClassifier.train(x_train)